 <table>
    <tr>
        <td>
           <img src='https://drive.google.com/uc?id=1V8z6gn4o_8NZqjnGCC4kiaMd10IxOFyF' height='200px' align='center'>
        </td>          
    </tr>
</table>

+ **Alunos:** 
    + Alex Felipe Barbosa
    + Danilo Silva Andrade

---

### 1.) DATA ACQUISITION

---

+ Instalação e Declaração das bibliotecas

In [33]:
# Install libraries first before declaring them
# !pip install pandas
# !pip install numpy
# !pip install plotly

# Declare libraries
import pandas as pd
import numpy as np
import plotly

+ conexão ao database ( temporariamente local )

In [34]:
# Search the path
dir_db = ('C:\\Users\\da04257\\Downloads\\tcc\\dir_db\\')

# Atributte the path database and search file
archive = ('dir_db//DadosEquipamentos_v3.csv')

+ leitura inicial da base de dados

In [35]:
# Read and select columns
dataset = pd.read_csv(archive, sep=';')

# Days of harvest
days_eqpto = dataset.groupby('CD_EQUIPAMENTO').DT_APONTAMENTO.nunique() # 264 e 257 ofic.
days = (days_eqpto / 2)

# Hours of day
hours = 24

# Hours of harvest
hh = (days.mean() * hours)

# Print first 5 rows of dataset
dataset.head(5)

,DT_APONTAMENTO,ANO,MES,CD_EQUIPAMENTO,CD_ATIVIDADE,NM_ATIVIDADE,FG_ATIVIDADE_PRODUTIVA,FG_ATIVIDADE_MANUTENCAO,VA_DURACAO,VA_TEMPO_MOTOR_LIGADO,VA_TEMPO_MOTOR_OCIOSO,VA_TEMPO_TRABALHO,VA_ABASTECIMENTO,VA_KM_TOTAL,VA_KM_TRABALHO,VA_VELOCIDADE_MEDIA_TR,VA_AREA_TRABALHO
0,25/03/2019,2019,3,225049,2001,Arremate,1,0,2818,2770,12,116,0,"0,85895","0,09467","2,93824","0,01419"
1,25/03/2019,2019,3,225049,2000,Manobras,1,0,562,554,0,1,0,"0,21556","0,00065","2,32143","0,0001"
2,25/03/2019,2019,3,225049,1942,Desembuchamento,1,0,11,11,0,0,0,"0,00556",0,0,0
3,25/03/2019,2019,3,225049,1939,Aguardando Transbordo,0,0,5424,44,0,0,0,0,0,0,0
4,25/03/2019,2019,3,225049,1903,Aguardando Caminhao,0,0,426,61,0,0,0,"0,00067",0,0,0


---

### 2.) DATA PREPARATION

---

##### 2.1) Análise Descritiva

In [20]:
# Display the following:
# - type of database
# - size of dataframe and yours columns
# - datatypes the columns
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370720 entries, 0 to 370719
Data columns (total 17 columns):
DT_APONTAMENTO             370720 non-null object
ANO                        370720 non-null int64
MES                        370720 non-null int64
CD_EQUIPAMENTO             370720 non-null int64
CD_ATIVIDADE               370720 non-null int64
NM_ATIVIDADE               370720 non-null object
FG_ATIVIDADE_PRODUTIVA     370720 non-null int64
FG_ATIVIDADE_MANUTENCAO    370720 non-null int64
VA_DURACAO                 370720 non-null int64
VA_TEMPO_MOTOR_LIGADO      370720 non-null int64
VA_TEMPO_MOTOR_OCIOSO      370720 non-null int64
VA_TEMPO_TRABALHO          370720 non-null int64
VA_ABASTECIMENTO           370720 non-null object
VA_KM_TOTAL                370720 non-null object
VA_KM_TRABALHO             370720 non-null object
VA_VELOCIDADE_MEDIA_TR     370720 non-null object
VA_AREA_TRABALHO           370720 non-null object
dtypes: int64(10), object(7)
memory usage: 48.1+ 

In [21]:
# Display the following:
# - main statistical measures of the dataframe
dataset.describe()

,ANO,MES,CD_EQUIPAMENTO,CD_ATIVIDADE,FG_ATIVIDADE_PRODUTIVA,FG_ATIVIDADE_MANUTENCAO,VA_DURACAO,VA_TEMPO_MOTOR_LIGADO,VA_TEMPO_MOTOR_OCIOSO,VA_TEMPO_TRABALHO
count,370720.000000,370720.000000,370720.000000,370720.000000,370720.000000,370720.000000,370720.000000,370720.000000,370720.000000,370720.000000
mean,2019.548819,7.151252,225583.491301,1862.796237,0.487028,0.136294,3061.386950,1779.801910,7.981517,1097.818634
std,0.497612,2.433261,265.904515,268.015907,0.499832,0.343101,7100.234059,3898.127068,61.282112,3661.130785
min,2019.000000,3.000000,225049.000000,1130.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2019.000000,5.000000,225395.000000,1905.000000,0.000000,0.000000,82.000000,7.000000,0.000000,0.000000
50%,2020.000000,7.000000,225724.000000,1943.000000,0.000000,0.000000,671.000000,80.000000,0.000000,0.000000
75%,2020.000000,9.000000,225880.000000,2000.000000,1.000000,0.000000,2894.000000,1292.000000,0.000000,16.000000
max,2020.000000,12.000000,225885.000000,2002.000000,1.000000,1.000000,89055.000000,41168.000000,8465.000000,40484.000000


##### 2.2) Transformação e Limpeza dos dados
+ Transformar, renomear, agrupar, etc.

In [22]:
# data grouping and transformation in hours
dt_group = dataset.groupby([dataset['DT_APONTAMENTO'], 'ANO', 'MES', 'CD_EQUIPAMENTO', 'CD_ATIVIDADE', 'NM_ATIVIDADE', 'FG_ATIVIDADE_PRODUTIVA', 'FG_ATIVIDADE_MANUTENCAO'])[['VA_TEMPO_MOTOR_LIGADO', 'VA_TEMPO_MOTOR_OCIOSO', 'VA_DURACAO']].sum() / 60 / 60

# rounding columns dataset
dt_group = dt_group.round(2)

# Display first 5 rows of dataset update
# dt_group.head(5)
# ------ For Check: db_resumo.query('DT_APONTAMENTO == "07/03/2019"')

# Saving the dataset for Day / Equipto / Atividade
dt_group.to_csv('dir_db\dt_group.csv', encoding='utf-8', index=True)

# Search the database
archive = ('dir_db/dt_group.csv')

# Read and select columns
dataset = pd.read_csv(archive, sep=',')

# Print first 10 rows of dataset
dataset.head(5)

# Decode Activities
dataset['FG_ATIVIDADE_PRODUTIVA'] = dataset.FG_ATIVIDADE_PRODUTIVA.replace(0,"Improdutiva")
dataset['FG_ATIVIDADE_PRODUTIVA'] = dataset.FG_ATIVIDADE_PRODUTIVA.replace(1,"Produtiva")


### 6. Agrupando diversos datasets: por Ano / Mes / Atividade

dt_year_month = dataset.groupby([dataset['ANO'], dataset['MES'], 'CD_ATIVIDADE', 'NM_ATIVIDADE', 'FG_ATIVIDADE_PRODUTIVA', 'FG_ATIVIDADE_MANUTENCAO'])[['VA_TEMPO_MOTOR_LIGADO', 'VA_TEMPO_MOTOR_OCIOSO', 'VA_DURACAO']].sum()
# rounding columns dataset
dt_year_month = dt_year_month.round(2)


# Groupby Year
dt_year = dataset.groupby([dataset['ANO'], 'CD_ATIVIDADE', 'NM_ATIVIDADE', 'FG_ATIVIDADE_PRODUTIVA', 'FG_ATIVIDADE_MANUTENCAO'])[['VA_DURACAO']].sum()
# rename column of database
#dt_year = dt_year.rename(columns={"FG_ATIVIDADE_PRODUTIVA": "TIPO_ATIVIDADE"})
# rounding columns dataset
dt_year = dt_year.round(2)

# Saving the datasets
dt_year_month.to_csv('dir_db\\dt_year_month.csv', encoding='utf-8', index=True)
dt_year.to_csv('dir_db\\dt_year.csv', encoding='utf-8', index=True)

# Search the database
archive_year_month = ('dir_db/dt_year_month.csv')
archive_year = ('dir_db/dt_year.csv')

# Read and select columns
dt_year_month = pd.read_csv(archive_year_month, sep=',')
dt_year = pd.read_csv(archive_year, sep=',')

# Define sum hours 2019
horas_tot_19 = dt_year.query('ANO == 2019')
horas_tot_19 = (horas_tot_19['VA_DURACAO'].sum() * 1)

# Define sum hours 2020
horas_tot_20 = dt_year.query('ANO == 2020')
horas_tot_20 = (horas_tot_20['VA_DURACAO'].sum() * 1)

dt_year['%_HORAS_ATV'] = np.where(dt_year['ANO'] == 2019, round(dt_year['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_year['VA_DURACAO'] / horas_tot_20 * 100,2))


In [23]:
import plotly.figure_factory as ff

dt_year_graf = dt_year.query('ANO == 2019')

# Rename column of database
dt_year_graf = dt_year_graf.rename(columns={"VA_DURACAO": "HORAS_ATV", "FG_ATIVIDADE_PRODUTIVA": "TIPO_ATIVIDADE"}, errors="raise")

# Select columns of database
dt_year_graf = dt_year_graf[['ANO', 'TIPO_ATIVIDADE', 'NM_ATIVIDADE', '%_HORAS_ATV']]

# Sort Values in database
dt_year_graf = dt_year_graf.sort_values(['ANO', 'TIPO_ATIVIDADE', '%_HORAS_ATV'], ascending=False)

# Plot database
fig =  ff.create_table(dt_year_graf, height_constant=20)
print(' / ')
print(' / ')
print(' / Relação % de Horas Produtivas e Improdutivas por Ano e Tipos de Atividades (2019)')
fig.show()

 / 
 / 
 / Relação % de Horas Produtivas e Improdutivas por Ano e Tipos de Atividades (2019)


In [24]:
import plotly.figure_factory as ff

dt_year_graf = dt_year.query('ANO == 2020')

# Rename column of database
dt_year_graf = dt_year_graf.rename(columns={"VA_DURACAO": "HORAS_ATV", "FG_ATIVIDADE_PRODUTIVA": "TIPO_ATIVIDADE"}, errors="raise")

# Select columns of database
dt_year_graf = dt_year_graf[['ANO', 'TIPO_ATIVIDADE', 'NM_ATIVIDADE', '%_HORAS_ATV']]

# Sort Values in database
dt_year_graf = dt_year_graf.sort_values(['ANO', 'TIPO_ATIVIDADE', '%_HORAS_ATV'], ascending=False)

# Plot database
fig =  ff.create_table(dt_year_graf, height_constant=20)
print(' / ')
print(' / ')
print(' / Relação % de Horas Produtivas e Improdutivas por Ano e Tipos de Atividades (2020)')
fig.show()

 / 
 / 
 / Relação % de Horas Produtivas e Improdutivas por Ano e Tipos de Atividades (2020)


---

### 3. Explanatory Analysis

---

In [25]:
########## FATOR CLIMÁTICO ##########

atv_1 = dt_year.query("CD_ATIVIDADE == 1910")  # Chuva / Escesso de Umidade
atv_2 = dt_year.query("CD_ATIVIDADE == 1921")  # Outros Fatores Climáticos
atv_3 = dt_year.query("CD_ATIVIDADE == 1935")  # Incêndio

dt_clima = pd.concat([atv_1, atv_2, atv_3], ignore_index=True)
dt_clima['CD_ATIVIDADE'] = "Climatica"
dt_clima['%_HORAS_ATV'] = np.where(dt_clima['ANO'] == 2019, round(dt_clima['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_clima['VA_DURACAO'] / horas_tot_20 * 100,2))

dt_clima = dt_clima.groupby([dt_clima['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_HORAS_ATV']].sum()

# Saving the datasets
dt_clima.to_csv('dir_db\\dt_clima.csv', encoding='utf-8', index=True)

# import
archive_dt_clima = ('dir_db/dt_clima.csv')
# read
dt_clima = pd.read_csv(archive_dt_clima, sep=',')
dt_clima = dt_clima.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe 
dt_clima['HC'] = hh
dt_clima['DIAS'] = 0
dt_clima['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_clima['DIAS'] = round(dt_clima['%_HORAS_ATV'] * dt_clima['HC'] / 24 / 100,0)
dt_clima['DIF_DIAS'] = dt_clima['DIAS'][0] - dt_clima['DIAS'][1]





########## FATOR MANUTENÇÃO ###########

atv_1 = dt_year.query("FG_ATIVIDADE_MANUTENCAO == 1")  # Todas paradas por manutenção

dt_manut = pd.concat([atv_1], ignore_index=True)
dt_manut['CD_ATIVIDADE'] = "Manutencao"
dt_manut['%_HORAS_ATV'] = np.where(dt_manut['ANO'] == 2019, round(dt_manut['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_manut['VA_DURACAO'] / horas_tot_20 * 100,2))

dt_manut = dt_manut.groupby([dt_manut['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_HORAS_ATV']].sum()

# Saving the datasets
dt_manut.to_csv('dir_db\\dt_manut.csv', encoding='utf-8', index=True)

# import
archive_dt_manut = ('dir_db/dt_manut.csv')
# read
dt_manut = pd.read_csv(archive_dt_manut, sep=',')
dt_manut = dt_manut.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe
dt_manut['HC'] = hh
dt_manut['DIAS'] = 0
dt_manut['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_manut['DIAS'] = round(dt_manut['%_HORAS_ATV'] * dt_manut['HC'] / 24 / 100,0)
dt_manut['DIF_DIAS'] = dt_manut['DIAS'][0] - dt_manut['DIAS'][1]





########## FATOR OPERACIONAL ##########

# Filter databases for defining dt_oper
atv_1 = dt_year.query("FG_ATIVIDADE_MANUTENCAO == 0 and FG_ATIVIDADE_PRODUTIVA == 'Improdutiva'")  # Todas paradas diferentes de manutenção e improdutivas
atv_2 = atv_1.query("CD_ATIVIDADE != 1910")  # Chuva / Escesso de Umidade
atv_3 = atv_2.query("CD_ATIVIDADE != 1921")  # Outros Fatores Climáticos
atv_4 = atv_3.query("CD_ATIVIDADE != 1935")  # Incêndio

# reserve for last analisy
dt_oper_final = pd.concat([atv_4], ignore_index=True)
dt_oper = pd.concat([atv_4], ignore_index=True)

# rewriting column
dt_oper['CD_ATIVIDADE'] = "Operacional"

# define condition for %_HORAS_ATV
dt_oper['%_HORAS_ATV'] = np.where(dt_oper['ANO'] == 2019, round(dt_oper['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_oper['VA_DURACAO'] / horas_tot_20 * 100,2))

# group by ANO and CD_ATIVIDADE
dt_oper = dt_oper.groupby([dt_oper['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_HORAS_ATV']].sum()

# define sum hours dt_oper in 2019
dt_oper_19 = dt_oper.query("ANO == 2019")
dt_oper_19 = dt_oper_19.sort_values(['%_HORAS_ATV'], ascending=False)
va_duracao_19 = round(dt_oper_19['VA_DURACAO'][0],2)

# define sum hours dt_oper in 2020
dt_oper_20 = dt_oper.query("ANO == 2020")
dt_oper_20 = dt_oper_20.sort_values(['%_HORAS_ATV'], ascending=False)
va_duracao_20 = round(dt_oper_20['VA_DURACAO'][0],2)

# Saving the datasets
dt_oper.to_csv('dir_db\\dt_oper.csv', encoding='utf-8', index=True)

# import
archive_dt_oper = ('dir_db/dt_oper.csv')
# read
dt_oper = pd.read_csv(archive_dt_oper, sep=',')
dt_oper = dt_oper.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe
dt_oper['HC'] = hh
dt_oper['DIAS'] = 0
dt_oper['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_oper['DIAS'] = round(dt_oper['%_HORAS_ATV'] * dt_oper['HC'] / 24 / 100,0)
dt_oper['DIF_DIAS'] = dt_oper['DIAS'][0] - dt_oper['DIAS'][1]




########## FATOR PRODUTIVO ##########

dt_produtivo = dt_year.query("FG_ATIVIDADE_MANUTENCAO == 0 and FG_ATIVIDADE_PRODUTIVA == 'Produtiva'")  # Todas paradas diferentes de manutenção e improdutivas

dt_produtivo = pd.concat([dt_produtivo], ignore_index=True)

dt_produtivo['CD_ATIVIDADE'] = "Produtiva"
dt_produtivo['%_HORAS_ATV'] = np.where(dt_produtivo['ANO'] == 2019, round(dt_produtivo['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_produtivo['VA_DURACAO'] / horas_tot_20 * 100,2))

dt_produtivo = dt_produtivo.groupby([dt_produtivo['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_HORAS_ATV']].sum()

# Saving the datasets
dt_produtivo.to_csv('dir_db\\dt_produtivo.csv', encoding='utf-8', index=True)

# import
archive_dt_produtivo = ('dir_db/dt_produtivo.csv')
# read
dt_produtivo = pd.read_csv(archive_dt_produtivo, sep=',')
dt_produtivo = dt_produtivo.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe
dt_produtivo['HC'] = hh
dt_produtivo['DIAS'] = 0
dt_produtivo['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_produtivo['DIAS'] = round(dt_produtivo['%_HORAS_ATV'] * dt_produtivo['HC'] / 24 / 100,0)
#dt_produtivo['DIF_DIAS'] = dt_produtivo['DIAS'][0] - dt_produtivo['DIAS'][1]

# Concat dataframe (Oficial Result)
df_result = pd.concat([dt_clima,dt_manut,dt_oper,dt_produtivo])
df_result = df_result.drop(columns=['HC'])

# rename column of database
df_result = df_result.rename(columns={"CD_ATIVIDADE": "CATEGORIA"})



In [26]:
import plotly.figure_factory as ff

df_result = df_result[['ANO', 'CATEGORIA', '%_HORAS_ATV']] #'DIAS', 'DIF_DIAS'
df_result['%_HORAS_ATV'] = round(df_result['%_HORAS_ATV'],2)
fig =  ff.create_table(df_result, height_constant=20)
print('')
print(' % DE HORAS PARADAS POR ANO E CATEGORIA DE ATIVIDADE')
print('')
fig.show()
print('Obs.: % calculado com base no total de horas paradas nas Safras 2019 e 2020')
print('')


 % DE HORAS PARADAS POR ANO E CATEGORIA DE ATIVIDADE



Obs.: % calculado com base no total de horas paradas nas Safras 2019 e 2020



In [27]:
print('TOTAL DE HORAS PARADAS EM ATIVIDADES OPERACIONAIS')

print('')
print('Safra 2019: ', + va_duracao_19)
print('Safra 2020: ', + va_duracao_20)
print('')

TOTAL DE HORAS PARADAS EM ATIVIDADES OPERACIONAIS

Safra 2019:  31136.32
Safra 2020:  29227.4



In [28]:
print('')
print('CALCULANDO % DE HORAS PARADAS APENAS PARA AS ATIVIDADES DA CATEGORIA OPERACIONAL ...')

# atribute databases
dt_oper = dt_oper_final

# rewriting column
dt_oper['CATEGORIA'] = "Operacional"

# define NEW condition for %_HORAS_ATV
dt_oper['%_HORAS_ATV'] = np.where(dt_oper['ANO'] == 2019, round(dt_oper['VA_DURACAO'] / va_duracao_19 * 100,2) , round(dt_oper['VA_DURACAO'] / va_duracao_20 * 100,2))

# group by ANO, CATEGORIA, CD_ATIVIDADE and NM_ATIVIDADE
dt_oper = dt_oper.groupby([dt_oper['ANO'], 'CATEGORIA', 'CD_ATIVIDADE', 'NM_ATIVIDADE'])[['VA_DURACAO', '%_HORAS_ATV']].sum()

# Saving the datasets
dt_oper.to_csv('dir_db\\dt_oper_final.csv', encoding='utf-8', index=True)

# import
archive_dt_oper_final = ('dir_db/dt_oper_final.csv')

# read
dt_oper_final = pd.read_csv(archive_dt_oper_final, sep=',')
#dt_oper_final = dt_oper_final.drop(['VA_DURACAO'], axis=1)

dt_oper_final = dt_oper_final.sort_values(['%_HORAS_ATV'], ascending=False)
dt_oper_final = dt_oper_final.drop(columns=['CD_ATIVIDADE', 'VA_DURACAO'])

print('')


CALCULANDO % DE HORAS PARADAS APENAS PARA AS ATIVIDADES DA CATEGORIA OPERACIONAL ...



In [29]:
import plotly.figure_factory as ff


dt_oper_final_19 = dt_oper_final.query("ANO == 2019")
dt_oper_final_19 = dt_oper_final_19[['ANO', 'CATEGORIA', 'NM_ATIVIDADE', '%_HORAS_ATV']]
dt_oper_final_19['%_HORAS_ATV'] = round(dt_oper_final_19['%_HORAS_ATV'],2)

dt_oper_final_19 = dt_oper_final_19.head(5)

fig =  ff.create_table(dt_oper_final_19, height_constant=20)
print('')
print(' % DE HORAS PARADAS OPERACIONAIS POR ANO E ATIVIDADE')
print('')
fig.show()
print('Obs.: Esta análise considera como 100% apenas o total de horas das atividades operacionais da safra 2019.')
print('')


 % DE HORAS PARADAS OPERACIONAIS POR ANO E ATIVIDADE



Obs.: Esta análise considera como 100% apenas o total de horas das atividades operacionais da safra 2019.



In [30]:
import plotly.figure_factory as ff


dt_oper_final_20 = dt_oper_final.query("ANO == 2020")
dt_oper_final_20 = dt_oper_final_20[['ANO', 'CATEGORIA', 'NM_ATIVIDADE', '%_HORAS_ATV']]
dt_oper_final_20['%_HORAS_ATV'] = round(dt_oper_final_20['%_HORAS_ATV'],2)

dt_oper_final_20 = dt_oper_final_20.head(5)

fig =  ff.create_table(dt_oper_final_20, height_constant=20)
print('')
print(' % DE HORAS PARADAS OPERACIONAIS POR ANO E ATIVIDADE')
print('')
fig.show()
print('Obs.: Esta análise considera como 100% apenas o total de horas das atividades operacionais da safra 2020.')
print('')


 % DE HORAS PARADAS OPERACIONAIS POR ANO E ATIVIDADE



Obs.: Esta análise considera como 100% apenas o total de horas das atividades operacionais da safra 2020.



---

### 4.) VISUALIZATION AND COMMUNICATION

---

In [31]:
import plotly.graph_objects as go

dt_oper_final_19 #database
dt_oper_final_20 #database

fig = go.Figure(data=[
    go.Bar(name='2019',
           x=dt_oper_final_19['NM_ATIVIDADE'],
           y=dt_oper_final_19['%_HORAS_ATV'],
           marker_color='MediumSlateBlue',
           text=dt_oper_final_19['%_HORAS_ATV'],
           textposition='auto'),
    
    go.Bar(name='2020',
           x=dt_oper_final_20['NM_ATIVIDADE'],
           y=dt_oper_final_20['%_HORAS_ATV'],
           marker_color='LightSkyBlue',
           text=dt_oper_final_19['%_HORAS_ATV'],
           textposition='auto')
])

fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')

# Change the bar mode
fig.update_layout(barmode='group',
                  title='% de Horas Improdutivas da Categoria Operacional - Ranking das 5 maiores',                  
                  xaxis_tickfont_size=11,
                  yaxis=dict(
                        title='% (percentual)',
                        titlefont_size=14,
                        tickfont_size=14,
                    ),)
fig.show()